#Analysis of Gaussian IL-PAH investigation

This document is intended to outline and discuss the outcome of the pygauss API that I have buit on top of the chemlab package to analyse output from Gaussian. In particular, for the analysis of Ionic-Liquids and Graphenic structures.

In [ ]:
import pygauss.molecule as pg
%matplotlib inline
from IPython.display import display
import os; home = os.path.expanduser("~")
file_path = os.path.join(home, 'Dropbox', 'Mres_Project', 
                         'gaussian', 'for_analysis')

##IL Cation Analysis

The basic function of the API is to take in data files ouput from Gaussian for a particular system and analyse their outcome in terms of:
- Geometry alignment, and
- Electronic distribution

In [ ]:
cation = pg.Molecule(file_path, 
                    init_fname='CJS1_emim_-_init.com', 
                    opt_fname='CJS1_emim_-_6-311+g-d-p-_gd3bj_opt_.log',
                    freq_fname='CJS1_emim_-_6-311+g-d-p-_gd3bj_freq_.log',
                    nbo_fname='CJS1_emim_-_6-311+g-d-p-_gd3bj_pop-nbo-full-_.log')

Initial analysis can be made of the computed optimisation, confirming that it was successful, that it is a saddle point within the potential energy surface and its energy (basis set dependant).

In [ ]:
print 'Basis Set: {0} ({1} functions)'.format(cation.get_basis_descript(), cation.get_basis_funcs())
print 'Optimised:', cation.is_optimised()
print 'Conformer:', cation.is_conformer()
print 'Energy:', cation.get_optimisation_E(units='kJmol-1')
cation.plot_IRfreqs()
#cation.plot_optimisation_E(units='kJmol-1')

An initial qualitative analysis of the geometry can be made.

In [ ]:
display(cation.show_initial(ball_stick=True, zoom=2,
                    rotations=[[0, 0, 90], [90, 90, 0], [225, 45, 45]], axis_length=0.5))
display(cation.show_optimisation(ball_stick=True, zoom=2,
                    rotations=[[0, 0, 90], [90, 90, 0], [225, 45, 45]], axis_length=0.5))

As can be seen above, the geometry is generally aligned arbitrarily to the standard axes. Therefore, an algorithm has been included to align the geometry to a plane defined by three atoms. This allows geometry to be more easily compared and will also come in helpful later for calculating relative orientations. 

In [ ]:
cation.add_alignment_atoms(3, 2, 1)
display(cation.show_highlight_atoms([[3], [2], [1]], optimised=False, ball_stick=True, zoom=2,
                            rotations=[[0, 0, 90], [90, 90, 0], [5, 55, 45]], axis_length=0.5))
display(cation.show_highlight_atoms([[3], [2], [1]], optimised=True, ball_stick=True, zoom=2,
                            rotations=[[0, 0, 90], [90, 90, 0], [5, 55, 45]], axis_length=0.5))

For $C_nC_1imidazolium$ based cations, a key variable in its geometric orientation is the $C_n$ chains angle relative to the $imidazolium$ ring.

In [ ]:
cation.show_highlight_atoms([[1, 4], [9, 10]], optimised=False, 
                            ball_stick=True, zoom=4, rotations=[[0, 0, 90]])

This can be computed as below:

In [ ]:
print 'Initial chain angle:', cation.calc_dihedral_angle([1,4,9,10], optimisation=False)
print 'Optimised chain angle:', cation.calc_dihedral_angle([1,4,9,10], optimisation=True)

Another measure which can be visaulised is the relative atomic charge, as calculated from Natural Bond Orbital (NBO) analysis. 

In [ ]:
cation.show_nbo_charges(ball_stick=True, zoom=4, rotations=[[0, 0, 90], [90, 180, 0], [45, 45, 90]], 
                        minval=-1, maxval=1, axis_length=0.3)

By taking either the positive or negatively charged atoms, we can weight their coordinates by charge and then calculate a mean *charge center* to give a simplistic metric of charge distribution. This is given in polar coordinates, relative to the ring plane.

In [ ]:
print 'Positive charge center; r = {0} nm, theta = {1}, phi = {2}'.format(
            *cation.calc_nbo_charge_center(3, 2, 1, atoms=range(1, 20)))

##IL Cation-Anion Pair Analysis

Three anions will be assessed, with each $C_nC_1imidazolium$, simplistically representing three different geometries;
- a point ($Cl$),
- a ball ($BF_4$),
- and a ball and chain ($EtSO_4$).

In [ ]:
display(pg.Molecule(file_path, 'CJS1_emim-cl_B_init.com').show_initial(ball_stick=True))
display(pg.Molecule(file_path, 'CJS1_emim-bf_B_init.com').show_initial(ball_stick=True))
display(pg.Molecule(file_path, 'CJS1_emim-etso_B_init.com').show_initial(ball_stick=True))

The minimum distance of the ions can be calculated as an initial metric.

In [ ]:
il_pair = pg.Molecule(file_path, 
            init_fname='CJS1_emim-cl_T_init.com', 
            opt_fname='CJS1_emim-cl_T_6-311+g-d-p-_gd3bj_opt-modredundant_unfrz.log',
            freq_fname='CJS1_emim-cl_T_6-311+g-d-p-_gd3bj_freq_unfrz.log',
            nbo_fname='CJS1_emim-cl_T_6-311+g-d-p-_gd3bj_pop-nbo-full-_unfrz.log',
            alignto=[3, 2, 1])

In [ ]:
print 'Minimum cation-anion distance: ', il_pair.calc_min_dist(
                                            range(1, 20), [20]), 'nm'

The position of the anion about the cation can also be calculated, in polar coordintes relative to the cation ring. 

In [ ]:
il_pair.show_highlight_atoms([[3, 2, 1], [20]], ball_stick=True, 
                             rotations=[[180, 0, 90], [270, 0, 0]], zoom=2 , axis_length=0.5)

In [ ]:
print 'Anion position; r = {0} nm, theta = {1}, phi = {2}'.format(
            *il_pair.calc_polar_coords_from_plane(3, 2, 1, 20))

We can also use **Second Order Perturbation Theory** analysis to identify potential regions of charge transfer
(LP = Lone Pair, BD* = Anti-Bonding).

In [ ]:
print il_pair.calc_SOPT_bonds(min_energy=20.)
il_pair.show_SOPT_bonds(min_energy=20., rotations=[[180, 0, 90], [270, 90, 0]], width=700)

###Conformer Analysis

In [ ]:
import pygauss.analysis as ag
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display
import os; home = os.path.expanduser("~")
file_path = os.path.join(home, 'Dropbox', 'Mres_Project', 
                         'gaussian', 'for_analysis')

In [ ]:
analysis = ag.Analysis(file_path)
analysis.add_run({'Cation':'emim'},
                init_fname='CJS1_emim_-_init.com', 
                opt_fname='CJS1_emim_-_6-311+g-d-p-_gd3bj_opt_.log',
                freq_fname='CJS1_emim_-_6-311+g-d-p-_gd3bj_freq_.log',
                nbo_fname='CJS1_emim_-_6-311+g-d-p-_gd3bj_pop-nbo-full-_.log')
analysis.add_run({'Cation':'omim'},
                init_fname='CJS1_omim_-_init.com', 
                opt_fname='CJS1_omim_-_6-311+g-d-p-_gd3bj_opt-modredundant_unfrz.log',
                freq_fname='CJS1_omim_-_6-311+g-d-p-_gd3bj_freq_unfrz.log',
                nbo_fname='CJS1_omim_-_6-311+g-d-p-_gd3bj_pop-nbo-full-_unfrz.log')
for mol in analysis.yield_mol_images(): display(mol)

In [ ]:
df, errors = analysis.add_runs(headers=['Cation', 'Anion', 'Initial', 'Stage'], 
            values=[['omim', 'emim'], ['cl', 'bf', 'etso'],
            ['B', 'BE', 'BM', 'F', 'FE', 'FM', 'T'], ['final']],
            init_pattern='CJS1_{0}-{1}_{2}_init.com',
            opt_pattern='CJS1_{0}-{1}_{2}_6-311+g-d-p-_gd3bj_opt-modredundant_unfrz.log',
            freq_pattern='CJS1_{0}-{1}_{2}_6-311+g-d-p-_gd3bj_freq_unfrz.log',
            nbo_pattern='CJS1_{0}-{1}_{2}_6-311+g-d-p-_gd3bj_pop-nbo-full-_unfrz.log',
            ipython_print=True)
print 'Read Errors:', errors

In [ ]:
mols = analysis.yield_mol_images(mtype='initial', 
                                      filters={'Cation':'emim', 'Anion':'cl'}, 
                                      align_to=[3,2,1], 
                                      rotations=[[0, 0, 90], [-90, 90, 0]],
                                      axis_length=0.5)
for mol in mols: display(mol)

In [ ]:
analysis.add_basic_properties()
analysis.remove_non_conformers()

In [ ]:
analysis.add_mol_property('Energy (au)', 'get_optimisation_E', units='hartree')
analysis.get_table(head=5)

In [ ]:
def add_subset_properties(filters, diatoms, cation, anion, anion_pivot):
    analysis.add_mol_property_subset('Cation Chain, $\\psi$', 'calc_dihedral_angle', 
                                     filters=filters, args=[diatoms])
    analysis.add_mol_property_subset('Cation Charge', 'calc_nbo_charge', 
                                     filters=filters, args=[cation])
    analysis.add_mol_property_subset(['Cation Charge, $r$', 
                                      'Cation Charge, $\\theta$', 
                                      'Cation Charge, $\\phi$'], 
                                   'calc_nbo_charge_center', filters=filters,
                                     args=[3, 2, 1], kwargs={'atoms':cation})
    analysis.add_mol_property_subset('Anion Charge', 'calc_nbo_charge', 
                                     filters=filters, args=[anion])
    analysis.add_mol_property_subset(['Anion-Cation $r$', 'Anion-Cation $\\theta$', 'Anion-Cation $\\phi$'], 
                                   'calc_polar_coords_from_plane', filters=filters,
                                     args=[3, 2, 1, anion_pivot])
    analysis.add_mol_property_subset('Anion-Cation $d_{min}$', 'calc_min_dist', 
                                     filters=filters, args=[cation, anion])    
    
add_subset_properties({'Cation':'emim'}, [1, 4, 9, 10], range(1, 20), range(20, 32), 20)
add_subset_properties({'Cation':'emim','Anion':'bf'}, [1, 4, 9, 10], range(1, 20), range(20, 25), 20)
add_subset_properties({'Cation':'emim','Anion':'cl'}, [1, 4, 9, 10], range(1, 20), [20], 20)
add_subset_properties({'Cation':'omim','Anion':'cl'}, [1, 4, 9, 17], range(1, 16)+range(17, 39), [16], 16)
add_subset_properties({'Cation':'omim','Anion':'bf'}, [1, 4, 9, 21], range(1, 16)+range(22, 43), range(16, 22), 16)
add_subset_properties({'Cation':'omim','Anion':'etso'}, [1, 4, 9, 28], range(1, 16)+range(28, 50), range(16, 28), 16)

In [ ]:
analysis.get_table(columns=[0,1,2]+range(8,19), filters={'Anion':['cl']},
                   row_index=['Cation', 'Anion', 'Initial'], column_index=['Cation', 'Anion', 'Anion-Cation'])

RadViz is a way of visualizing multi-variate data. It is based on a simple spring tension minimization algorithm. Basically you set up a bunch of points in a plane. In our case they are equally spaced on a unit circle. Each point represents a single attribute. You then pretend that each sample in the data set is attached to each of these points by a spring, the stiffness of which is proportional to the numerical value of that attribute (they are normalized to unit interval). The point in the plane, where our sample settles to (where the forces acting on our sample are at an equilibrium) is where a dot representing our sample will be drawn. Depending on which class that sample belongs it will be colored differently.

In [ ]:
analysis.plot_radviz_comparison('Anion', columns=range(8, 19), 
                                filters={'Cation':'emim', 'Anion':['cl', 'bf', 'etso']})
plt.gcf().set_size_inches(7,7)

In [ ]:
analysis.plot_radviz_comparison('Anion', columns=range(8, 19), 
                                filters={'Cation':'omim', 'Anion':['cl', 'bf', 'etso']})
plt.gcf().set_size_inches(7,7)

The KMeans algorithm clusters data by trying to separate samples in n groups of equal variance, minimizing a criterion known as the inertia or within-cluster sum-of-squares. This algorithm requires the number of clusters to be specified. It has been used across a large range of application areas in many different fields.

In [ ]:
kwargs = {'mtype':'optimised', 'align_to':[3,2,1], 
            'rotations':[[0, 0, 90], [-90, 90, 0]],
            'axis_length':0.5}
def show_groups(df):
    for cat, gf in df.groupby('Category'):
        print 'Category {0}:'.format(cat)
        mols = analysis.yield_mol_images(rows=gf.index.tolist(), **kwargs)
        for mol, row in zip(mols, gf.index.tolist()): 
            print '(row {0})'.format(row)
            display(mol)

In [ ]:
show_groups(analysis.calc_kmean_groups('Anion', 'cl', 5, columns=range(8, 17), 
                                filters={'Cation':'emim'}))

In [ ]:
show_groups(analysis.calc_kmean_groups('Anion', 'bf', 2, columns=range(8, 17), 
                                filters={'Cation':'emim'}))                              

In [ ]:
show_groups(analysis.calc_kmean_groups('Anion', 'etso', 2, columns=range(8, 17), 
                                filters={'Cation':'emim'}))

In [ ]:
show_groups(analysis.calc_kmean_groups('Anion', 'etso', 3, columns=range(8, 17), 
                                filters={'Cation':'omim'}))

##Coronene Ion Conformers

In [ ]:
import pygauss.analysis as ag
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display
import os; home = os.path.expanduser("~")
file_path = os.path.join(home, 'Dropbox', 'Mres_Project', 
                         'gaussian', 'for_analysis')

In [ ]:
analysis2 = ag.Analysis(file_path)
df, errors = analysis2.add_runs(headers=['Surface', 'Ion', 'Initial'], 
            values=[['coro'], ['cl', 'bf'],
            ['P', 'T-M', 'T-S']],
            init_pattern='CJS1_{0}-{1}_{2}_init.com',
            ipython_print=True)
print 'Read Errors:', errors
df, errors2 = analysis2.add_runs(headers=['Surface', 'Ion', 'Initial'], 
            values=[['coro'], ['etso'],
            ['P-O', 'P-P', 'T-M-O', 'T-M-P', 'T-S-O', 'T-S-PI', 'T-S-PO']],
            init_pattern='CJS1_{0}-{1}_{2}_init.com',
            ipython_print=True)
print 'Read Errors:', errors, errors2
df, errors3 = analysis2.add_runs(headers=['Surface', 'Ion', 'Initial'], 
            values=[['coro'], ['emim'],
            ['S-C37C38-P-O', 'S-C39-P-O', 'S-O-E', 'S-O-N40N44',
            'T-C4-C37C38-P', 'T-C4-C39-P', 'T-C9-P-O', 'T-C9-P-P',
            'T-M-C37C38-P', 'T-M-C39-P', 'T-M-P-O', 'T-M-P-P']],
            init_pattern='CJS1_{0}-{1}_{2}_init.com',
            ipython_print=True)
print 'Read Errors:', errors, errors2, errors3

In [ ]:
run=0
for mol in analysis2.yield_mol_images(mtype='initial', align_to=[4,7,9],
                                rotations=[[0,0,0], [90,0,0]], axis_length=0.5):
    pos = analysis2.get_table().Initial.iloc[run]; run+=1
    key={'P':'Parallel', 'T':'Top', 'M':'Middle', 'S':'Side', 'O':'Orthogonal',
        'PO':'Parallel (out)', 'PI':'Parallel (in)'}
    print ", ".join( [ key.get(abbrev,abbrev) for abbrev in pos.split('-')] )
    display(mol)